# CÁLCULO DE VIGA PROTENDIDA - VP02

**OBRA:** Carnetti Open Mall

**Cliente:** JD Construções e Pré-Fabricados

**Data:** 23/02/2022

**Cálculo:** Gabriel


Todas as grandezas apresentadas possuem como unidades base kN [força] e cm [comprimento].

In [1]:
import handcalcs.render
import pandas as pd
from math import floor,ceil,sqrt,exp
from numpy import log

def calcpsi(tensao):
    if (tensao <= 0.5):
        return 0
    elif (tensao <= 0.6):
        extra = tensao - 0.5
        adicional = 13*extra
        return (0+adicional)/100
    elif (tensao <= 0.7):
        extra = tensao - 0.6
        adicional = 12*extra
        return (1.3+adicional)/100
    else:
        extra = tensao - 0.7
        adicional = 10*extra
        return (2.5+adicional)/100
    
pi = 3.141592653

# Informações Iniciais:

## Definição do EPS:

In [2]:
h_eps = 70
e_eps = 12

## Geometria da Viga:

In [3]:
%%render
# Geometria da Viga:

h = 120
b = 30
Area = b*h - e_eps*h_eps
Inercia = b*h**3/12 - e_eps*h_eps**3/12
W_i = Inercia/(h/2)
W_s = Inercia/(h/2)
e_p = 55
Vao = 2391

## Número de seções para dividir o vão:
# Escolher número impar
nS = 101

<IPython.core.display.Latex object>

## Propriedades dos Materiais:

In [4]:
%%render
# Propriedades dos Materiais:

f_ck = 4
f_ctm = 0.03*(10*f_ck)**(2/3)
f_ptk = 190
f_pyd = 148.7
f_yk = 50
f_cj = 3.2

E_s = 21000
E_p = 20000

<IPython.core.display.Latex object>

In [5]:
%%render 2 short

f_c = f_ck*0.85/1.4
E_ci = 560*(f_ck*10)**(1/2)
E_cs = E_ci*(0.8+0.2*f_ck/8)

E_cit0 = (f_cj/f_ck)**(1/2)*E_ci
E_cst0 = E_cit0*(0.8+0.2*f_cj/8)

f_yd = f_yk/1.15
sigma_p0 = 0.8*f_ptk

<IPython.core.display.Latex object>

## Carregamentos:

In [6]:
%%render 2 short

Q_t0 = 0.000025*Area
Q_tF = 0.156



<IPython.core.display.Latex object>

## Esforços no Centro do Vão:

In [7]:
%%render 2 short
M_F = (Q_tF)*Vao**2*0.125
M_0 = (Q_t0)*Vao**2*0.125
V_F = (Q_tF)*Vao/2

<IPython.core.display.Latex object>

# Deteminação da Força de Protensão:

## Limites de Tensão:

In [8]:
%%render short

sigma_ct0 = -1.2*0.03*(f_cj*10)**(2/3)
sigma_cc0 = 0.7*f_cj
sigma_ctF = -1.2*0.03*(f_ck*10)**(2/3)
sigma_ccF = 0.5*f_ck

r_0 = 0.95
r_F = 0.80

<IPython.core.display.Latex object>

## Cálculo da Força de Protensão:

In [9]:
%%render short 2

P_1 = (sigma_cc0+M_0/W_i)/(r_0*(1/Area + e_p/W_i)) 
P_2 = (sigma_ct0-M_0/W_s)/(r_0*(1/Area - e_p/W_s)) 
P_3 = (sigma_ctF+M_F/W_i)/(r_F*(1/Area + e_p/W_i))
P_4 = (sigma_ccF-M_F/W_s)/(r_F*(1/Area - e_p/W_s))


##Adotar valores entre:
P_min = max(P_3, P_4)
P_max = min(P_1, P_2)


<IPython.core.display.Latex object>

## Escolha da Protensão:

In [10]:
%%render 2
# Força de Protensão Adotada:

P = P_max
phi_1 = 1.52
A_p1 = 1.40

## Número de cordoalhas para adotar:
n = floor(P/(A_p1*sigma_p0))

<IPython.core.display.Latex object>

## Ancoragem das Cordoalhas:

### Comprimento de Ancoragem Básico:

In [11]:
%%render short
f_pyd = 0.9* f_ptk / 1.15

eta_p1 = 1.2

# Valores de Eta_p1:
#     
#     1.0 para fios lisos
#     1.2 para cordoalhas de 3 e 7 fios
#     1.5 para fios dentados

eta_p2 = 1.0

# Valores de Eta_p2:
#     
#     1.0 para situação de boa aderência
#     0.7 para situação de má aderência

f_ctd = (0.7*0.03*(f_cj*10)**(2/3))/1.4
f_bpd = eta_p1 * eta_p2 * f_ctd

l_bp = (7*phi_1)/36 * f_pyd/f_bpd

<IPython.core.display.Latex object>

### Cálculo do comprimento de Transferência:

In [12]:
%%render short

## para fios lisos ou dentados:
l_bpt1 = 0.7 * l_bp * sigma_p0/f_pyd

## para cordoalhas de 3 e 7 fios:
l_bpt2 = 0.5 * l_bp * sigma_p0/f_pyd

## para transferência não gradual, os valores devem ser multiplicados por 1.25
## valor adotado:

l_bpt = 1.25 * l_bpt2

l_p = (h**2 + (0.6*l_bpt)**2)**(0.5)


<IPython.core.display.Latex object>

#  Esforços nas Seções:

## Momento Fletor e Cortante:

In [13]:
esforcos = []
for i in range(3):
    esforcos.append( [0]*(nS) )
    
# primeiro vetor contem X (cm)
# segundo vetor contem Momentos (kNcm)
# terceiro vetor contém cortante (kN)

for i in range(nS):
    esforcos[0][i] = round(i*(Vao/(nS-1)),1)
    esforcos[1][i] = round( 0.5*Q_tF*esforcos[0][i]*(Vao-esforcos[0][i]) ,1)
    esforcos[2][i] = round( 0.5*Q_tF*(Vao-2*esforcos[0][i]), 1 )
    
tabela = pd.DataFrame(esforcos)
tabela = tabela.transpose()
tabela.rename(columns={0:'Pos (cm)',1:'M (kNcm)',2:'V (kN)'},inplace = True)
display(tabela)

,Pos (cm),M (kNcm),V (kN)
0,0.0,0.0,186.5
1,23.9,4412.7,182.8
2,47.8,8736.4,179.0
3,71.7,12970.9,175.3
4,95.6,17116.3,171.6
...,...,...,...
96,2295.4,17116.3,-171.6
97,2319.3,12970.9,-175.3
98,2343.2,8736.4,-179.0
99,2367.1,4412.7,-182.8


# Escalonamento da Protensão:

## Limites de Tensão na Seção S1:

São testados os limites de tensão para t0:

Limitação pela borda superior:

In [14]:
%%render short

M_s1 = round( 0.5*Q_t0*esforcos[0][1]*(Vao-esforcos[0][1]) ,1)
P_1 = (sigma_cc0+M_s1/W_i)/(r_0*(1/Area + e_p/W_i))
P_2 = (sigma_ct0-M_s1/W_s)/(r_0*(1/Area - e_p/W_s))


<IPython.core.display.Latex object>

In [15]:
%%render 1

P_s1 = min(P_1 , P_2)

n_s1 = floor(P_s1/(A_p1*sigma_p0))


<IPython.core.display.Latex object>

## Escolha do Número de Cordoalhas:

In [16]:
%%render

# Vetor com o número de cordoalhas em cada ponto de escalonamento:

qntCordoalhas = (5,10)

# Vetor com a excentricidade da protensão em cada ponto de escalonamento:

e_p_i = [55,52.5]

<IPython.core.display.Latex object>

# Esforços Devidos À Protensão:

In [17]:
A_p = []

for i in range(len(qntCordoalhas)):
    A_p.append(qntCordoalhas[i] * A_p1)

W_cp = []

for i in range(len(qntCordoalhas)):
    W_cp.append(round(Inercia/e_p_i[i],0))
    
alpha_p0 = E_p / E_cst0

K_p0 = []
for i in range(len(qntCordoalhas)):
    K_p0.append(round(alpha_p0*A_p[i]*(1/Area+e_p_i[i]/W_cp[i]),4))
    
sigma_p = []
for i in range(len(qntCordoalhas)):
    sigma_p.append(round(sigma_p0/(1+K_p0[i]),2))
    
N_p = []
for i in range(len(qntCordoalhas)):
    N_p.append(round(-sigma_p[i]*A_p[i],1))
    
M_p = []
for i in range(len(qntCordoalhas)):
    M_p.append(round(N_p[i]*e_p_i[i],1))
    
tabela1 = pd.DataFrame([A_p, W_cp, K_p0, sigma_p, N_p, M_p])
tabela1 = tabela1.transpose()
tabela1.rename(columns={0:'Ap',1:'Wcp',2:'Kp0',3:'Sigma,p',4:'Np',5:'Mp'},inplace = True)
display(tabela1)

,Ap,Wcp,Kp0,"Sigma,p",Np,Mp
0,7.0,72309.0,0.0564,143.88,-1007.2,-55396.0
1,14.0,75752.0,0.1060,137.43,-1924.0,-101010.0


# Escalonamento da Força de Protensão:

In [18]:
M_g = []
x1 = []
x2 = []
x1_anc = []
x2_anc = []

for i in range(len(qntCordoalhas)):
    M_g.append(round( W_s*( (N_p[i]/Area) -(M_p[i]/W_s) + sigma_ct0),2) )
        
for i in range(len(qntCordoalhas)):
    #Bhaskara:
    x1.append(round( (Vao - ( Vao**2 - 8*M_g[i]/Q_t0 )**0.5)/2  ,2))
    x2.append(round( (Vao + ( Vao**2 - 8*M_g[i]/Q_t0 )**0.5)/2  ,2))
    
for i in range(len(qntCordoalhas)):
    x1_anc.append(x1[i]-round(l_bpt,2))
    x2_anc.append(x2[i]+round(l_bpt,2))
    if x1_anc[i] < 0:
        x1_anc[i] = 0
    if x2_anc[i] > Vao:
        x2_anc[i] = Vao
    

tabela2 = pd.DataFrame([M_g, x1,x2, x1_anc,x2_anc])
tabela2 = tabela2.transpose()
tabela2.rename(columns={0:'Mg',1:'x1',2:'x2',3:'x1-lbpt',4:'x2+lbpt'},inplace = True)
display(tabela2)

,Mg,x1,x2,x1-lbpt,x2+lbpt
0,7156.0,90.15,2300.85,0.00,2391.00
1,30752.4,462.12,1928.88,307.36,2083.64


# Perdas Progressivas da Força de Protensão:

## Fluência do Concreto:

### Fluência Rápida Irreversível

In [19]:
%%render

# VER ITEM 12.3.3 da NBR6118:2014
# Valores para adotar:
# s = 0.38 para concreto de cimento CP-III e CP-IV
# s = 0.25 para concreto de cimento CP-I e CP-II
# s = 0.20 para concreto de cimento CP-V
s = 0.20
t_0 = 10
t_F = 10000


beta_0 = exp(s*(1-(28/t_0)**0.5))
beta_F = exp(s*(1-(28/t_F)**0.5))

phi_a = 0.8*(1 - (beta_0/beta_F))

<IPython.core.display.Latex object>

### Fluência Lenta Irreversível:

In [20]:
%%render 2

U = 55 #umidade média do ar

phi_c1 = 4.45 - 0.035*U # para abatimentos entre 5cm e 9cm

gamma = 1 + exp(-7.8 + 0.1 * U)

u_ar = ( 2*h + 2*b )

h_fic = gamma*(2*Area)/u_ar

phi_c2 = (42+h_fic)/(20+h_fic)

phi_fF = phi_c1*phi_c2

h_fm = h_fic/100 # espessura fictícia em metros

A_beta_f = 42*h_fm**3 - 350*h_fm**2 + 588*h_fm + 113
B_beta_f = 768*h_fm**3 - 3060*h_fm**2 + 3234*h_fm - 23
C_beta_f = -200*h_fm**3 + 13*h_fm**2 + 1090*h_fm + 183
D_beta_f = 7579*h_fm**3 - 31916*h_fm**2 + 35343*h_fm + 1931

beta_f_t0 = (t_0**2 + A_beta_f*t_0 + B_beta_f )/(t_0**2 + C_beta_f*t_0 + D_beta_f)
beta_f_tF = (t_F**2 + A_beta_f*t_F + B_beta_f )/(t_F**2 + C_beta_f*t_F + D_beta_f) 

<IPython.core.display.Latex object>

### Fluência Lenta Reversível:

In [21]:
%%render short

phi_dF = 0.4

beta_d = (t_F-t_0+20)/(t_F-t_0+70)

<IPython.core.display.Latex object>

### Fluência Total:

In [22]:
%%render 2
phi_t = phi_a + phi_fF*(beta_f_tF - beta_f_t0) + phi_dF * beta_d

<IPython.core.display.Latex object>

### Retração do Concreto:

In [23]:
%%render 2

epsilon_1s = (-8.09 + U/15 - U**2/2284 + U**3/133765 - U**4/7608150)*10**-4

epsilon_2s = (33+2*h_fic)/(20.8 + 3*h_fic)

A_beta_s = 40  
B_beta_s = 116*h_fm**3 - 282*h_fm**2 + 220*h_fm - 4.8  
C_beta_s = 2.5*h_fm**3 - 8.8*h_fm + 40.7  
D_beta_s = -75*h_fm**3 + 585*h_fm**2 + 496*h_fm - 6.8  
E_beta_s = -169*h_fm**4 + 88*h_fm**3 + 584*h_fm**2 - 39*h_fm + 0.8  

beta_s_t0 = ((t_0/100)**3 + A_beta_s*(t_0/100)**2 +  B_beta_s*(t_0/100))/((t_0/100)**3 + C_beta_s*(t_0/100)**2 +  D_beta_s*(t_0/100) + E_beta_s)

beta_s_tF = ((t_F/100)**3 + A_beta_s*(t_F/100)**2 +  B_beta_s*(t_F/100))/((t_F/100)**3 + C_beta_s*(t_F/100)**2 +  D_beta_s*(t_F/100) + E_beta_s)

epsilon_cs = epsilon_1s*epsilon_2s*(beta_s_tF - beta_s_t0)


<IPython.core.display.Latex object>

## Relaxação do Aço de Protensão:

In [24]:
relax = []
for i in range(7):
    relax.append([0]*nS)

for i in range (nS):
    relax[0][i] = i+1
    relax[1][i] = esforcos[0][i] #posição x da seção
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            relax[2][i] = round(sigma_p[j],2)
            break
    relax[3][i] = round(relax[2][i]/f_ptk,3)
    relax[4][i] = round(calcpsi(relax[3][i]),3)
    relax[5][i] = round(relax[4][i]*2.5,3)
    relax[6][i] = round(-log(1-relax[5][i]),3)
    
    
tabela3 = pd.DataFrame(relax)
tabela3 = tabela3.transpose()
tabela3.rename(columns={0:'Seção',1:'Pos',2:'Sigma,p',3:'Sigma,p/fptk',4:'\u03A81000',5:'\u03A8,inf',6:'X,inf'},inplace = True)
display(tabela3)

,Seção,Pos,"Sigma,p","Sigma,p/fptk",Ψ1000,"Ψ,inf","X,inf"
0,1.0,0.0,143.88,0.757,0.031,0.077,0.08
1,2.0,23.9,143.88,0.757,0.031,0.077,0.08
2,3.0,47.8,143.88,0.757,0.031,0.077,0.08
3,4.0,71.7,143.88,0.757,0.031,0.077,0.08
4,5.0,95.6,143.88,0.757,0.031,0.077,0.08
...,...,...,...,...,...,...,...
96,97.0,2295.4,143.88,0.757,0.031,0.077,0.08
97,98.0,2319.3,143.88,0.757,0.031,0.077,0.08
98,99.0,2343.2,143.88,0.757,0.031,0.077,0.08
99,100.0,2367.1,143.88,0.757,0.031,0.077,0.08


## Cálculo das Perdas Progressivas:

In [25]:
progressivas = []

for i in range(4):
    progressivas.append([0] * nS)
    
alpha_p = E_p/E_cs
    
for i in range(nS):
    momento = esforcos[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    progressivas[0][i] = round(Npi/Area + Mpi/Wcpi + momento/Wcpi,2)
    K_p = alpha_p*A_pp*(1/Area + e_pp/Wcpi)
    progressivas[1][i] = round( (( progressivas[0][i]*phi_t/E_cs + epsilon_cs - sigma_pi/E_p*relax[6][i] ) / (1 + relax[6][i] + K_p*(1+0.5*phi_t)) )*E_p,2)
    progressivas[2][i] = -round(progressivas[1][i]*A_pp  ,2)
    progressivas[3][i] = round(progressivas[2][i]*e_pp ,1)
    
    

tabela4 = pd.DataFrame(progressivas)
tabela4 = tabela4.transpose()
tabela4.rename(columns={0:'Sigma,c,t0',1:'\u0394Sigma,p',2:'\u0394Np',3:'\u0394Mp'},inplace = True)
display(tabela4)

,"Sigma,c,t0","ΔSigma,p",ΔNp,ΔMp
0,-1.13,-37.62,263.34,14483.7
1,-1.07,-36.53,255.71,14064.0
2,-1.01,-35.43,248.01,13640.6
3,-0.95,-34.33,240.31,13217.0
4,-0.89,-33.24,232.68,12797.4
...,...,...,...,...
96,-0.89,-33.24,232.68,12797.4
97,-0.95,-34.33,240.31,13217.0
98,-1.01,-35.43,248.01,13640.6
99,-1.07,-36.53,255.71,14064.0


# Tensões Normais nas Bordas:

## Bordas Superiores:

In [26]:
tensaosup = []

for i in range(3):
    tensaosup.append([0]*nS)
    
for i in range(nS):
    tensaosup[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaosup[1][i] = round( -abs(Npx/Area) + abs(Mpx/W_s) - abs(esforcos[1][i]/W_s) ,3)
    if tensaosup[1][i] > -sigma_ccF:
        tensaosup[2][i] = "OK"
    else:
        tensaosup[2][i] = "--> NAO OK <--"
    
    

tabela5 = pd.DataFrame(tensaosup)
tabela5 = tabela5.transpose()
tabela5.rename(columns={0:'Pos (cm)',1:'Sigma,s',2:'OK'},inplace = True)
display(tabela5)

,Pos (cm),"Sigma,s",OK
0,0.0,0.348,OK
1,23.9,0.285,OK
2,47.8,0.223,OK
3,71.7,0.163,OK
4,95.6,0.104,OK
...,...,...,...
96,2295.4,0.104,OK
97,2319.3,0.163,OK
98,2343.2,0.223,OK
99,2367.1,0.285,OK


## Bordas Inferiores:

In [27]:
tensaoinf = []

for i in range(3):
    tensaoinf.append([0]*nS)
    
for i in range(nS):
    tensaoinf[0][i] = relax[1][i]
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pi = sigma_p[j]
            Npi = N_p[j]
            Mpi = M_p[j]
            Wcpi = W_cp[j]
            e_pp = e_p_i[j]
            A_pp = A_p[j]
            break
    Npx = Npi + progressivas[2][i]
    Mpx = Mpi + progressivas[3][i]
    tensaoinf[1][i] = round( -abs(Npx/Area) - abs(Mpx/W_i) + abs(esforcos[1][i]/W_i) ,3)
    if tensaoinf[1][i] < -sigma_ctF:
        tensaoinf[2][i] = "OK"
    else:
        tensaoinf[2][i] = "--> NAO OK <--"
    
tabela6 = pd.DataFrame(tensaoinf)
tabela6 = tabela6.transpose()
tabela6.rename(columns={0:'Pos (cm)',1:'Sigma,i',2:'OK'},inplace = True)
display(tabela6)

,Pos (cm),"Sigma,i",OK
0,0.0,-0.887,OK
1,23.9,-0.829,OK
2,47.8,-0.773,OK
3,71.7,-0.719,OK
4,95.6,-0.665,OK
...,...,...,...
96,2295.4,-0.665,OK
97,2319.3,-0.719,OK
98,2343.2,-0.773,OK
99,2367.1,-0.829,OK


# Armaduras Passivas:

## Armadura da Cunha de Tração:

In [28]:
sigma_cs0 = -10000
pos = -1
for i in range (nS):
    if sigma_cs0 < tensaosup[1][i]:
        sigma_cs0 = tensaosup[1][i]
        pos = i

sigma_ci0 = tensaoinf[1][pos]

print("Seção crítica de cálculo: seção",pos)

Seção crítica de cálculo: seção 0


In [29]:
%%render 2

x = sigma_cs0/(sigma_cs0 - sigma_ci0) * h

S_x = b*x**2/2

R_ct = sigma_cs0/x*S_x

A_st = R_ct/25.00

phi_st = 1.6

N_st = ceil(A_st/((phi_st/2)**2*pi))

<IPython.core.display.Latex object>

## Pré-Alongamento de Leito:

In [30]:
prealong = []

for i in range(5):
    prealong.append([0] * nS)
    
for i in range(nS):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
                sigma_pw = sigma_p[j]
                Npw = N_p[j]
                Mpw = M_p[j]
                Wcpw = W_cp[j]
                break
    prealong[0][i] = Npw + progressivas[2][i]
    prealong[1][i] = Mpw + progressivas[3][i]
    prealong[2][i] = round(prealong[0][i]/Area + prealong[1][i]/Wcpw ,2)
    prealong[3][i] = round( sigma_pw + progressivas[1][i] - alpha_p*prealong[2][i] ,2)
    prealong[4][i] = round(prealong[3][i]/E_p,5)

tabela7 = pd.DataFrame(prealong)
tabela7 = tabela7.transpose()
tabela7.rename(columns={0:'Np,F',1:'Mp,F',2:'Sigma,cpF', 3:'Sigma,PN',4:'e,pn'},inplace = True)
display(tabela7)

,"Np,F","Mp,F","Sigma,cpF","Sigma,PN","e,pn"
0,-743.86,-40912.3,-0.84,111.53,0.00558
1,-751.49,-41332.0,-0.84,112.62,0.00563
2,-759.19,-41755.4,-0.85,113.78,0.00569
3,-766.89,-42179.0,-0.86,114.95,0.00575
4,-774.52,-42598.6,-0.87,116.10,0.00580
...,...,...,...,...,...
96,-774.52,-42598.6,-0.87,116.10,0.00580
97,-766.89,-42179.0,-0.86,114.95,0.00575
98,-759.19,-41755.4,-0.85,113.78,0.00569
99,-751.49,-41332.0,-0.84,112.62,0.00563


## Armadura Passiva de Flexão:

![formulario](img/formulario.png)

In [31]:
d_p = [] # braço de alavanca da armadura ativa
d_s = [] # braço de alavanca da armadura passiva

for i in range(len(qntCordoalhas)):
    d_p.append(e_p_i[i] + h/2)
    d_s.append(d_p[i] - 7.5) # estimativa para ds
    

A_s = []
for i in range(6):
    A_s.append([0] * nS)
    
for i in range(1,nS-1):
    for j in range(len(qntCordoalhas)-1,-1,-1):
        if(relax[1][i] >= x1_anc[j] and relax[1][i] <= x2_anc[j]):
            sigma_pm = sigma_p[j]
            Npm = N_p[j]
            Mpm = M_p[j]
            Wcpm = W_cp[j]
            A_pm = A_p[j]
            d_sm = d_s[j]
            d_pm = d_p[j]
            break
    Rpt = f_pyd * A_pm
    a_bh = 1
    b_bh = -2.5*d_sm
    c_bh = (esforcos[1][i]*1.4 + Rpt*(d_sm-d_pm))/(0.32*b*f_c)
    x_1 = (-b_bh + (b_bh**2 - 4*a_bh*c_bh )**0.5)/(2*a_bh)
    x_2 = (-b_bh - (b_bh**2 - 4*a_bh*c_bh )**0.5)/(2*a_bh)
    A_s[0][i] = round(x_1,2)
    A_s[1][i] = round(x_2,2)
    if (x_1 > h or x_1 < 0):
        x = x_2
    else:
        x = x_1
    A_s[2][i] = round(x,2)
    A_s[3][i] = round(3.5/x*(d_sm-x),2)
    A_s[4][i] = round(prealong[4][i] + 3.5/x*(d_pm-x),2)
    A_s[5][i] = round((0.8*b*x*f_c-Rpt)/f_yd,2)
    if (A_s[5][i] < 0):
        A_s[5][i] = 0

tabela8 = pd.DataFrame(A_s)
tabela8 = tabela8.transpose()
tabela8.rename(columns={0:'x1',1:'x2', 2:'x',3:'eps,sx',4:'eps,p',5:'As'},inplace = True)
display(tabela8)

,x1,x2,x,"eps,sx","eps,p",As
0,0.00,0.00,0.00,0.00,0.00,0.0
1,269.01,-0.26,-0.26,-1452.32,-1553.39,0.0
2,268.04,0.71,0.71,527.93,565.01,0.0
3,267.09,1.66,1.66,222.81,238.60,0.0
4,266.15,2.60,2.60,141.00,151.09,0.0
...,...,...,...,...,...,...
96,266.15,2.60,2.60,141.00,151.09,0.0
97,267.09,1.66,1.66,222.81,238.60,0.0
98,268.04,0.71,0.71,527.93,565.01,0.0
99,269.01,-0.26,-0.26,-1452.32,-1553.39,0.0


# Dimensionamento ao Esforço Cortante:

## Verificação da Diagonal Comprimida:

In [32]:
%%render

V_d = V_F * 1.4
tau_wd = V_d/((b-e_eps)*d_sm)
tau_wu = 0.27*(1-f_ck/25)*f_ck/1.4


<IPython.core.display.Latex object>

In [33]:
if tau_wu > tau_wd:
    print("OK!")
else:
    print("NÃO OK!")

OK!


## Cálculo da Armadura de Corte:

### Armadura Mínima:

In [34]:
%%render short

rho_min = 0.2*f_ctm/f_yk
A_sws = rho_min*b

s = 20

A_sw_min = A_sws*s

nr = 2

phi_min = 2*(A_sw_min/(pi*nr))**0.5


<IPython.core.display.Latex object>

### Armadura Calculada:

In [35]:
i = int((nS+1)/2)
M_pf = prealong[1][i]
N_pf = prealong[0][i]

In [36]:
%%render

sigma_ciF = N_pf/Area + M_pf/W_i
M_0 = -sigma_ciF*W_i
M_d_max = 1.4*M_F

V_c0 = 0.6*f_ctd*b*d_sm
V_c = (1+M_0/M_d_max)*V_c0

V_sw = 1.4*V_F - V_c

A_sws = V_sw/(0.9*d_sm*f_yd)

s = 20
nr = 2
phi_min = 2*(A_sws/(pi*nr))**0.5

<IPython.core.display.Latex object>

## Armadura de Introdução da Força de Protensão:

In [37]:
N_p1 = -N_p[0]

In [38]:
%%render

A_swi = (0.25*N_p1)/25.00

<IPython.core.display.Latex object>

Esta armadura deve ser distribuída num trecho equivalente à 70% do comprimento de regularização.

In [39]:
%%render

l_d = l_p*0.7

<IPython.core.display.Latex object>